## An End-to-End Trainable Neural Network for Image-based Sequence Recognition and Its Application to Scene Text Recognition
https://arxiv.org/pdf/1507.05717

Note: Currently transformes based models dominate state-of-the-art benchmarks, but CRNN-CTC models not entirely irrelevant
 - a lightweight option, edge devices
 - if the OCR problem is simpler/not as complex

Before CRNN, image-based sequence recognition systems were
 - complex
 - multi-stage pipelines
 - requiring separate, isolated training for components:
    - feature extraction
        - HOG (Histogram of Oriented Gradients) --> detects shapes
        - SIFT Scale-Invariant Feature Transform --> detect keypoints that are robust to scale/rotation --> for recognizing objects even if scaled/rotated/different lighting
    - segmentation - dividing image into meaningful parts
        - separating individual characters
    - sequence modeling

This paper proposes CRNN - Convolutional Recurrent Neural Network that succussfully integrates feature extraction, sequence modeling, and transcription into a unified framework that can be trained end-to-end.

### Three components
1) Convolutional layers
    - DCNN Deek convolutional Neural Network process image and convert into a sequence of feature vectors
2) Recurrent layers
    - Deep Bidirectional Recurrent Neural Network Bi-RNN stacked on top.
    - Bidirectional allows model to leverage context from both the left and ride sides of a character
3) Transcription layer
    - Uses Connectionist Temporal Classification CTC
    - key thing that enablet entire system to be trained end-to-end,
    - key to handle variable length sequences 
    - **without needing explicit character segmentation**

### Key Properties
 - End-to-end trainable
 - Handles arbitrary length
    - since it does not require character segmentation or horizontal normalization
 - Lexicon-Free Recognition
    - since it directly outputs character sequences
    - not limited by predefined lexicon(dictionary) of words,
    - can recognize rare words or proper nouns (iPhone not in dictionary)
    - still can work with dictionaries: pick closest match from provided dictionary
 - Generality: was used in other domains
    - music score recognition


### Deeper dive - CTC
#### Connectionist Temporal Classification
Its not a neural network itself, its a loss function and a scoring mechanism
 - solves a critical problem in seq-to-seq learning: labelling unsegmented data

#### Alignment Challenge
 - When we have a sequence input (column-by-column scan of an image)
 - and a sequence output (transcribed text)
the lengths rarely match, and the alignment between them is unknown

Eg. handwriting & speech recognition
```yaml
Image: "heeelllooo" 
Truth: "hello"
(e spans more pixels than h)
```
Traditional appoach is to segment to individual characters first, then classify each segment


### Two Key Concepts
#### 1) The Blank Token ϵ
CTC introduces a special label, the **blank token** into the output alphabet. 
 - token is NOT whitespace
 - represents a LACK of label/character output at a given timestep
#### 2) Many-to-One Mapping
The layer before outputs a probability distribution over all characters (predictions) **plus** the blank token for every time step.

These predictions then undergo a **two-step collapsing process**
1) Remove duplicated labels **not separated by a blank**
2) Remove all remaining blank tokens.

```txt
h e e _ l _ l _ o o
(collapse repeats)
h e _ l _ l _ o
(remove blank tokens)(
hello
```
### The Loss Function: Summing all Possible Paths
```python
import torch
import torch.nn as nn

ctc_loss = nn.CTCLoss()
loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
```
For a given input X,
 - and a target label Y ("CAT")
 - there are hundreds/thousands in the network's output that collapses down to final correct label Y
    - CCAATT, _C_A_T, C_AA_TTT all map to CAT
 - CTC loss calculates probablility of target label Y |(given)| input X 
    - by summing probablilities of all valid paths that collapse to Y

$$ P(Y \mid X) = \sum_{\pi \in B^{-1}(Y)} P(\pi \mid X) $$
Where $ B^{-1}(Y) $ is the set of all paths $\pi$ that map to Y

The loss is then the negative log of this total probability.

Since the number of paths is huge, this is calculated efficiently using the **Forward-backward Algorithm**

But in summary, CTC allows an entire sequence model to be trained end-to-end 
 - requiring only a gound-truth transcript (Y)
 - without needing tedious, manual time-step alignment

#### During inference we don't have a label
In the paper it uses a simple greedy search, at any time step `t`, we simply select the token with the highest conditional probability. 
For how computationally undemanding it is, its a great choice.

##### Beam search
A less efficient alternative is using beam search. not as efficient but changes from:
 - sequence of most likely tokens to:
 - most likely sequence

The key idea is that selecting a token with smaller conditional probability NOW might lead to a higher conditional probability overall

because the conditional probability will change at the next time step depending on the token that is selected at the current time step.

Lastly we have the most computationally demanding **exhaustive search**. Beam and this is well explained here: https://d2l.ai/chapter_recurrent-modern/beam-search.html